# AEROSP 536 Electric Propulsion: Homework 1
Jason Chen

## Problem 3
### Part (a)
We need to check for inconsistencies in the data provided. First, we can check the efficiency equation:
$$
\begin{equation}
\eta = \frac{1}{2} \left( \frac{T^2}{\dot{m}} \right) \frac{1}{P_{in}}
\end{equation}
$$
against the matched exit condition thrust equation:
$$
F_t = \dot{m} u_e = \dot{m} (I_{sp} g_0)
$$
And subract the two to compute a residual. If the residual is significant, it means the table entries given are inconsistent.

In [7]:
import numpy as np

data = {
    'P_in'     : np.array([4.46, 1.26, 1.25, 3.26, 3.24]) * 1e3,
    'F_t'      : np.array([357.8, 248.5, 264.0, 92.7, 95.0]) * 1e-3,
    'F_t_range': np.array([  4.6,   3.3,   2.7,  2.7,  3.0]) * 1e-3,
    'eta'      : np.array([0.47, 0.39, 0.42, 0.44, 0.49]),
    'eta_range': np.array([0.01, 0.02, 0.03, 0.01, 0.02]),
    'isp'      : np.array([1196, 1086, 1132, 1163, 1235]),
    'isp_range': np.array([  15,   32,   33,   16,   16])
}
num_entries = 5
for i in range(num_entries):
    mdot_1 = data['F_t'][i] / (data['isp'][i] * 9.81)
    mdot_2 = (1/2) * (data['F_t'][i]**2) / (data['eta'][i] * data['P_in'][i])
    print(f"Residual mass flow rate for Row {i+1}: {mdot_1 - mdot_2:.2e} kg/s")


Residual mass flow rate for 0: -4.060e-08 kg/s
Residual mass flow rate for 1: -3.951e-05 kg/s
Residual mass flow rate for 2: -4.260e-05 kg/s
Residual mass flow rate for 3: 5.130e-06 kg/s
Residual mass flow rate for 4: 4.999e-06 kg/s


We can see that all of these magnitudes are extremely small, so it is not the source of the inconsistency.

### Part (b)
We're given a chart of $F_t$ v.s. $I_{sp}$ and lines of constant input power $P_{in}$ and mass flow rate of Xenon $\dot{m}$. Since the company evaluated their performance at $100 \textrm{ W}$ and $5.2 \textrm{ mN}$, we can read the chart:

![](/home/jasonyc/sugs/536_electric_prop/hw_1_p3b.png)

We see that the intersection approximately yields $\dot{V} = 12 \textrm{ SCCM}$ and $I_{sp} = 440 \textrm{ s}$. Converting the volumetric flow rate to a mass flow rate requires knowledge that a standard cubic centimeter per minute is actually a molar flow rate, but expressed as a volumetric flow rate at standard conditions. Assuming the U.S. standard conditions of 293 K and 101.3 kPa, we get the density of Xenon at these conditions (via NIST REFPROP) as $\rho_{STP} = 5.49 \textrm{ kg/m}^3$, and so $\dot{m}_{\textrm{Xe}} = \dot{V}_{SCCM}/\rho_{STP}$

Alternatively, if we don't want to guess the SCCM trace, we can also use Equation (2) to find $\dot{m}$ from the given thrust and $I_{sp}$ values, for which we get $\dot{m} = 1.2 \times 10^{-6} \textrm{kg/s}$. We can now plug into Equation (1) directly to get:

$$
\eta = \frac{1}{2} \left( \frac{T^2}{\dot{m}} \right) \frac{1}{P_{in}} = 11.2 \%
$$




## Problem 4
The scenario given is the extreme case where the propulsion system is a laser, where the reaction mass is photons.

### Part (a)
Thrust to power ratio given 100% efficiency is:
$$
\frac{T}{P_{in}} = \frac{2\eta}{u_e} = \frac{2}{c} \approx 6.67 \times 10^{-9} \textrm{ s/m}
$$

### Part (b)
Given that state-of-the-art EP systems can generate about 350 mN at 5 kW, how much power would it take for the laser propulsion system to generate the same thrust?

The inverse of what we found above can tell us (assuming a linear relationship between input power and thrust generation):
$$
(350 \textrm{ mN})\left( \frac{T}{P_{in}} \right)^{-1} = 52.5 \textrm{ MW}
$$

### Part (c)
I would not use laser propulsion. As seen above, the power requirement to operate laser propulsion at EP thrust levels (which is already very low, which contributes to a trade of transit time) is very high, which is over ~2 orders of magnitude more than the power produced by the solar arrays on the ISS. The thrust and power trade is simply too impractical, unless we are constructing an interstellar probe with a nuclear reactor.

## Problem 5
To solve this problem, we start with the definition of thrust, neglecting the pressure thrust term (as is common with most EP devices) and assuming constant density as the problem states:

$$
\vec{T} = -\dot{m} \vec{u}_{\textrm{eff}} = - \int_{A_e} \rho_e \vec{u}_e (\vec{u}_e \cdot \hat{n}) dA_e = - \rho_e \hat{n} \int_{A_e} (\vec{u}_e \cdot \hat{n}) (\vec{u}_e \cdot \hat{n}) dA_e
$$

From the definition of mass flow rate:

$$
\dot{m} = \rho A u = \rho_e \int_{A_e} (\vec{u_e} \cdot \hat{n}) dA_e
$$

We can rearrange:
$$
|\vec{u}_{\textrm{eff}}| = \frac{\int_{A_e} (\vec{u}_e \cdot \hat{n}) (\vec{u}_e \cdot \hat{n}) dA_e}{\int_{A_e} (\vec{u_e} \cdot \hat{n}) dA_e}
$$

Now, we can write the area integral as a double integral in cylindrical coordinates. Let's first tackle the denominator, i.e. mass flow rate. Recall that when integrating in cylindrical coordinates, we need to include the Jacobian $r$:

$$
\int_{A} (\vec{u} \cdot \hat{n}) dA = \int_{0}^{2\pi} \int_{0}^{R} (\vec{u} \cdot \hat{n}) r dr d\theta
$$

From the problem statement, we know that the given velocity function has a uniform divergence angle $\theta_D = 25$ as measured from the nozzle axis, meaning:

$$
\vec{u}(r) \cdot \hat{n} = |\vec{u}|(r) |\hat{n}| \cos{\theta_D}
$$

from the definition of a dot product, and thus:
$$
\int_{A} (\vec{u} \cdot \hat{n}) dA = \int_{0}^{2\pi} \int_{0}^{R} \left( u_0 e^{-r^2/R^2} \right) \cos{\theta_D} r dr d\theta
$$

When doing a u-substitution of $u = r^2$, we can rewrite:

$$
= \frac{1}{2}u_0 \cos{\theta_D} \int_{0}^{2\pi} \int_{r_1=0}^{r_2=R} e^{-u/R^2} du d\theta
$$

Further doing a substitution of $v = -u/R^2$ and taking care to rewrite the integral bounds:

$$
= -\frac{1}{2}u_0 R^2 \cos{\theta_D} \int_{0}^{2\pi} \int_{v_1=0}^{v_2=-1} e^{v} dv d\theta
$$

We can now easily compute the antiderivatives and solve the definite integral, which results in:
$$
\int_{A} (\vec{u} \cdot \hat{n}) dA = \frac{1}{2}u_0 R^2 \cos{\theta_D} \left( 1-e^{-1} \right) (2\pi)
$$

We can repeat this process for the integral in the numerator, which instead has an integrand of $(\vec{u} \cdot \hat{n})^2$. Work is not shown here for brevity, but the  was checked symbolically with Wolfram Mathematica to be:

$$
\int_{A} (\vec{u} \cdot \hat{n})(\vec{u} \cdot \hat{n}) dA = \frac{1}{2} u_0^2 R^2 (\cos{\theta_D})^2 \left( 1-e^{-2} \right) \pi
$$

Thus, the ratio of these (and therefore the effective exhaust velocity) is:
$$
|\vec{u}_{\textrm{eff}}| = \frac{\frac{1}{2} u_0^2 R^2 (\cos{\theta_D})^2 \left( 1-e^{-1} \right) \left( 1+e^{-1} \right) \pi}{\frac{1}{2}u_0 R^2 \cos{\theta_D} \left( 1-e^{-1} \right) (2\pi)} = \frac{1}{2} u_0 \cos{\theta_D} \left( 1+e^{-1} \right) \approx 1952.6 \textrm{ m/s}
$$

Comparing with the maximum local axial component, i.e. the maximum of the function $u_0 e^{-r^2/R^2} \cos{\theta_D}$, which by inspection occurs when $r=0$ since this is where $e^{r^2/R^2}$ is at its minimum, gives $2854.9 \textrm{ m/s}$. Thus, we can clearly see that simply computing the maximum component to use as the effective exhaust velocity will severly overestimate the actual performance of the engine.

## Problem 6
The Psyche mission makes use of four Maxar SPT-140 Hall thrusters that when operating at 4 kW have a thrust of 235 mN and a specific impulse of 1770 s each. The total impulse requirement for this mission is 13.6 MN-s.

### Part (a)
To find the required operating time and propellant mass given the total impulse requirement, we use the definition of specific impulse as the total impulse over the weight of propellant:

$$
I_{sp} = \frac{I_t}{m_p g_0}
$$

so $m_p = 783.2 \textrm{ kg}$. Then, we can use mass flow rate and the total propellant mass to find the needed burn time:

$$
t_b = \frac{m_p}{\dot{m}} = \frac{m_p}{F/(I_{sp}g_0)} = 16,076 \textrm{ hours}
$$

### Part (b)
We cannot assume here that the efficiency stays constant, but we can solve for total impulse through another avenue. We know that the mass of the propellant required stays constant from Part (a) assuming $I_{sp}$ is constant (which is a better assumption than power efficiency), meaning still, $m_p = 783.2 \textrm{ kg}$. Now, we can solve for the mass flow rate that must be supplied given the propellant mass and operating time:

$$
\dot{m} = \frac{m_p}{t_b} = \frac{783.2 \textrm{ kg}}{10362 \textrm{ hours}} = 2.1 \times 10^{-5} \textrm{ kg/s}
$$

Now we can find the thrust that this corresponds with via $F = \dot{m}(I_{sp}g_0) = 365 \textrm{ mN}$ and multiply by the known burn time to get the total impulse:

$$
I_t = F t_b = 13.599 \textrm{ MN-s}
$$

This result is extremely close to the requirement of $13.6 \textrm{ MN-s}$ and can be dismissed as rounding error, and the final answer is **yes, the requirement is satisfied.**

### Part (c)



### Part (d)


In [12]:
import pandas as pd

csv_path = 'Psyche_Thrust_Variation.csv'
df = pd.read_csv(csv_path)
thrust3 = df['Simulation 3'].values * 1e-3  # mN to N
thrust4 = df['Simulation 4'].values * 1e-3  # mN to N
time_s = df['Time (Hours)'].values * 3600   # hours to seconds

# Trapezoidal integration for total impulse (N-s) using np.trapezoid
impulse3 = np.trapezoid(thrust3, time_s)
impulse4 = np.trapezoid(thrust4, time_s)

print(f"Total impulse for Simulation 3: {impulse3:.2f} N-s")
print(f"Total impulse for Simulation 4: {impulse4:.2f} N-s")
print(f"Difference: {abs(impulse3 - impulse4):.2f} N-s")

Total impulse for Simulation 3: 2482704.00 N-s
Total impulse for Simulation 4: 2435103.00 N-s
Difference: 47601.00 N-s
